# Simulation experiment using noisy data

Run entire simulation experiment multiple times to generate confidence interval.  The simulation experiment can be found in ```functions/pipeline.py```

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines, utils

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [ ]:
# Read in config variables
config_file = os.path.abspath(os.path.join(os.getcwd(),"../../configs", "config_Human_sample.tsv"))
params = utils.read_config(config_file)

In [ ]:
# Load parameters
dataset_name = params["dataset_name"]
analysis_name = params["analysis_name"]
NN_architecture = params["NN_architecture"]
num_simulated_samples = params["num_simulated_samples"]
lst_num_experiments = params["lst_num_experiments"]
use_pca = params["use_pca"]
num_PCs = params["num_PCs"]
local_dir = params["local_dir"]

iterations = params["iterations"] 
num_cores = params["num_cores"]

In [ ]:
# Additional parameters
file_prefix = "Experiment"
corrected = False

In [3]:
# Input file
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,    
    "data",
    "input",
    "recount2_gene_normalized_data.tsv.xz")

In [4]:
# Output files
similarity_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name +"_sample_lvl_sim_similarity_uncorrected.pickle")

ci_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name +"_sample_lvl_sim_ci_uncorrected.pickle")

similarity_permuted_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    dataset_name +"_sample_lvl_sim_permuted")

In [5]:
# Run multiple simulations - corrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.sample_level_simulation_uncorrected)(i,
                                                       NN_architecture,
                                                       dataset_name,
                                                       analysis_name,
                                                       num_simulated_samples,
                                                       lst_num_experiments,
                                                       corrected,
                                                       use_pca,
                                                       num_PCs,
                                                       file_prefix,
                                                       normalized_data_file,
                                                       local_dir) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 49.2min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 49.3min remaining: 74.0min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 49.4min remaining: 32.9min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 49.5min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 49.5min finished


In [6]:
# permuted score
permuted_score = results[0][0]

In [7]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of experiments,,,,,
1,0.999993,0.999993,0.999993,0.999993,0.999993
2,0.900672,0.903726,0.906249,0.901311,0.902036
5,0.625569,0.634569,0.639156,0.626586,0.621498
10,0.203885,0.193566,0.210531,0.199162,0.193022
20,0.123870,0.147143,0.141927,0.134515,0.144320
50,0.151582,0.167269,0.169646,0.166350,0.161792
100,0.265945,0.285986,0.263318,0.268157,0.265751
250,0.579445,0.545324,0.550906,0.567772,0.578551
500,0.887753,0.907680,0.902347,0.907965,0.882172


In [8]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of experiments,
1,0.999993
2,0.902799
5,0.629476
10,0.200033
20,0.138355
50,0.163328
100,0.269831
250,0.564400
500,0.897584


In [9]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of experiments,
1,4.682448e-08
2,7.085721e-04
5,2.274404e-03
10,2.327964e-03
20,2.958627e-03
50,2.263295e-03
100,2.906639e-03
250,4.959685e-03
500,3.763535e-03


In [10]:
# Get confidence interval for each row (number of experiments)
err = std_scores*1.96

In [11]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of experiments,,
1,0.999993,0.999993
2,0.901410,0.904188
5,0.625018,0.633933
10,0.195471,0.204596
20,0.132556,0.144154
50,0.158892,0.167764
100,0.264134,0.275529
250,0.554679,0.574121
500,0.890207,0.904960


In [12]:
mean_scores

,score
number of experiments,
1,0.999993
2,0.902799
5,0.629476
10,0.200033
20,0.138355
50,0.163328
100,0.269831
250,0.564400
500,0.897584


In [13]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)
np.save(similarity_permuted_file, permuted_score)